# #121 Mesh cracks
<i>Mesh cracks is a term refering to unintended discontinuities in the mesh. These are typically caused when model geomtry overlaps rather than merges together. When two lines are coincident but unmerged, each line will have nodes, rather than a single row of nodes. This script will check the distance between adjacent nodes of selected lines and indicate pairs of nodes within a tolernace for review</i>
***

In [ ]:
import pandas as pd
import numpy as np
import itertools   
import time
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI import *
lusas = get_lusas_modeller()
if not lusas.existsDatabase():
    raise Exception("A model must be open before running this code")


Define the distance below which we want to detect adjacent nodes

In [ ]:
TOLERANCE = 0.05 # Distance between nodes which is suspicious, this should be smaller than the mesh size
APPROACH = 2     # Approach to be taken, 2 is recommended
EXTENT_OPTION = 1 # 0=all nodes in database, 1=nodes in current selection (recommended)

# In general it is best to select lines to be checked since it is at lines that mesh cracks form, it is not worth checking all nodes within surfaces and volumes
try:
    lusas.setVisible(False) # Setting Lusas Modeller invisible speeds up calls to teh LPI because the application doesnt have to respond to user events at the same time.
    match EXTENT_OPTION:
        case 0: 
            nodes = lusas.database().getObjects("Nodes")
        case 1:
            nodes = lusas.selection().getObjects("Nodes")
finally:
    lusas.setVisible(True)

## Approach 1, using itertools to generate the necessary comparison combinations

Note, the limit of 100 nodes because this code is very slow for large number of nodes

In [ ]:
if APPROACH == 1:
    if len(nodes) > 100:
        assert False, "This will take a very long time"

    start = time.time()
    no_comparisons = 0

    # Use itertools to generate the comparison of each node with each other node
    # No of comparisons will be n? where the ? indicates summation of all integers from n to 0
    for a, b in itertools.combinations(nodes, 2):
        no_comparisons+=1
        delta = np.sqrt( (a.getX() - b.getX())**2 + (a.getY() - b.getY())**2 + (a.getZ() - b.getZ())**2 )
        if delta < TOLERANCE:
            print(f"Nodes {a.getID()} and {b.getID()} are within {delta:.3f}")

    print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")


## Approach 2 - As 1 but reducing calls to the LPI

Same approach as above here but first we compile a list of node coordinates. 
This avoids many repeated calls to get node coordinates

In [ ]:
if APPROACH == 2:
    start = time.time()
    no_comparisons = 0

    positions  = np.array([[n.getX(), n.getY(), n.getZ(), n.getID()] for n in nodes])
    for a, b in itertools.combinations(positions, 2):
        no_comparisons+=1
        delta = np.sqrt( (a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2 )
        if delta < TOLERANCE:
            print(f"Nodes {a[3]} and {b[3]} are within {delta:.3f}")

    print(f"Comparison time = {time.time() - start:.2f} seconds for {no_comparisons} comparisons")

As you can see repeated calls to the LPI functions can have a dramatic affect on performance and should be avoided where posible 

## Approach 3 using numpy arrays - more complicated

In [ ]:
if APPROACH == 3:
    start = time.time()
    ids = np.array([n.getID() for n in nodes])
    positions  = np.array([[n.getX(), n.getY(), n.getZ()] for n in nodes])

    # Relative positions of each node 
    rel = positions[None, :, :] - positions[:, None, :]

    # distances of each node from the x,y,z relative offsets
    distances = np.sqrt(np.square(rel).sum(axis=2))


    for i in range(0, len(distances)):
        a = np.where(distances[i] < TOLERANCE)
        if len(a[0]) == 2 and a[0][0] != i:
            print(f"Nodes {ids[a[0][0]]} and {ids[a[0][1]]} are within {distances[i][a[0][0]]:.3f}")


    print(f"Comparison time = {time.time() - start:.2f} seconds using numpy arrays")